In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('data/matches.csv')

In [3]:
df

,competition,date,score,homeTeam,homeTeamCountry,homeTeamUrl,awayTeam,awayTeamCountry,awayTeamUrl,location,phase,attendance,competitonOriginal,season
0,CWC,1966-04-14,1-0,Celtic,Scotland,Celtic_F.C.,Liverpool,England,Liverpool_F.C.,"Celtic Park, Glasgow",semi-finals,76397,1965–66 European Cup Winners' Cup,1965-66
1,ICFC,1955-06-04,0-5,Basel XI,Switzerland,Basel,London XI,England,London_XI,"St. Jakob Stadium, Basel",group stage,NaN,1955–58 Inter-Cities Fairs Cup,1955-58
2,UCL,1955-09-04,3-3,Sporting CP,Portugal,Sporting_CP,Partizan,Socialist Federal Republic of Yugoslavia,FK_Partizan,"Estádio Nacional, Lisbon",first round,30000,1955–56 European Cup,1955-56
3,UCL,1955-09-07,6-3,Vörös Lobogó,Hungary,MTK_Budapest_FC,Anderlecht,Belgium,R.S.C._Anderlecht,"Népstadion, Budapest",first round,35000,1955–56 European Cup,1955-56
4,UCL,1955-09-08,0-2,Servette,Switzerland,Servette_FC,Real Madrid,Spain,Real_Madrid_CF,"Charmilles Stadium, Geneva",first round,7000,1955–56 European Cup,1955-56
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27321,UECL,2024-08-15,v,Larne,Northern Ireland,NaN,Ballkani,Kosovo,NaN,"Inver Park, Larne",third qualifying round,NaN,2024–25 UEFA Conference League qualifying phas...,2024-25
27322,UECL,2024-08-15,v,Hajduk Split,Croatia,NaN,Ružomberok,Slovakia,NaN,"Stadion Poljud, Split",third qualifying round,NaN,2024–25 UEFA Conference League qualifying phas...,2024-25
27323,EL,2024-08-15,v,Viktoria Plzeň,Czech Republic,NaN,Kryvbas Kryvyi Rih,Ukraine,NaN,"Doosan Arena, Plzeň",third qualifying round,NaN,2024–25 UEFA Europa League qualifying phase an...,2024-25
27324,UCL,NaN,Match unplayed,Derry City,Northern Ireland,Derry_City_F.C.,Anderlecht,Belgium,R.S.C._Anderlecht,NaN,first round,NaN,1965–66 European Cup,1965-66


In [4]:
df['city'] = df['location'].apply(lambda x: x.split(',')[-1].split('(')[0].strip() if isinstance(x, str) else x)

In [5]:
df['date'] = pd.to_datetime(df['date'], errors='coerce')

In [6]:
def count_same_or_higher_date_comp(group):
    group = group.sort_values('date')
    group['remaining_comp'] = group['date'].apply(lambda x: (group['date'] > x).sum())
    return group

In [7]:
remaining = df.groupby(['season','competition']).apply(count_same_or_higher_date_comp).reset_index(drop=True)

C:\Users\micha\AppData\Local\Temp\ipykernel_31240\3916731917.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  remaining = df.groupby(['season','competition']).apply(count_same_or_higher_date_comp).reset_index(drop=True)


In [8]:
df = df.merge(remaining[['competition','date','remaining_comp']])

In [21]:
df = df.drop_duplicates()

In [22]:
df.loc[df['season'] == '2024-25', 'remaining_comp'] = None

In [23]:
serious = ['groups','group stage','group a','group b','group c','group d','round of 32','round of 16','quarter-finals','semi-finals','final']

In [24]:
easternCountries = ['Hungary','Poland','Socialist Federal Republic of Yugoslavia','Romania','Czechoslovakia','Bulgaria','East Germany','Albania','Soviet Union','Commonwealth of Independent States','Slovenia','Estonia','Latvia','Ukraine','Russia','Lithuania','Georgia (country)','Moldova','Croatia','Belarus','Czech Republic','Federal Republic of Yugoslavia','North Macedonia','Armenia','Azerbaijan','Slovakia','Bosnia and Herzegovina','Kazakhstan','Serbia and Montenegro','Serbia','Montenegro','Kosovo']

In [25]:
easternGermanClubs = df[df['homeTeamCountry'] == 'East Germany']['homeTeamUrl'].drop_duplicates().to_list() + ['RB_Leipzig','1._FC_Union_Berlin','1._FC_Magdeburg','Hallescher_FC','FC_Rot-Wei%C3%9F_Erfurt']

In [26]:
df['phase'].drop_duplicates().to_list()

['semi-finals',
 'group stage',
 'first round',
 'quarter-finals',
 'final',
 'preliminary round',
 'knockout stage',
 'second round',
 nan,
 'third round',
 'qualifying round',
 'groups',
 'group c',
 'group b',
 'group d',
 'group a',
 'round of 16',
 'finals',
 'first qualifying round',
 'second qualifying round',
 'third qualifying round',
 'fourth round',
 'round of 32',
 'play-off round',
 'knockout round play-offs']

In [27]:
east = df[df['homeTeamCountry'].isin(easternCountries) | df['homeTeamUrl'].isin(easternGermanClubs)]
east2 =  df[df['homeTeamCountry'].isin(easternCountries) | df['homeTeamUrl'].isin(easternGermanClubs) | df['awayTeamCountry'].isin(easternCountries) | df['awayTeamUrl'].isin(easternGermanClubs)]
four = df[df['homeTeamCountry'].isin(['Germany','West Germany','Italy','Spain','England'])]

In [28]:
east.groupby('homeTeam')['remaining_comp'].min().nsmallest(50)

homeTeam
Alashkert                 0.0
CFR Cluj                  0.0
CSKA Sofia                0.0
Dinamo Tbilisi            0.0
Dinamo Zagreb             0.0
Dnipro Dnipropetrovsk     0.0
Dynamo Kyiv               0.0
Ferencváros               0.0
Legia Warsaw              0.0
Magdeburg                 0.0
OFK Beograd               0.0
Partizan                  0.0
Rapid București           0.0
Red Star Belgrade         0.0
Riga                      0.0
Rubin Kazan               0.0
Ruch Chorzów              0.0
Shakhtar Donetsk          0.0
Sileks                    0.0
Slovan Bratislava         0.0
Slovan Liberec            0.0
Steaua București          0.0
Tiraspol                  0.0
Union Berlin              0.0
Vojvodina                 0.0
Zenit Saint Petersburg    0.0
Zorya Luhansk             0.0
Ararat-Armenia            1.0
Banik Ostrava             1.0
Budućnost Podgorica       1.0
CSKA Moscow               1.0
Celje                     1.0
Dinamo București          1.0
D

In [16]:
venku = df[(df['awayTeamCountry'].str.contains('Cze',na=False)) & (df['competition'] == 'UCL')].groupby(['awayTeam','season'])['remaining_comp'].min().groupby('awayTeam').first().nsmallest(20)
doma = df[(df['homeTeamCountry'].str.contains('Cze',na=False)) & (df['competition'] == 'UCL')].groupby(['homeTeam','season'])['remaining_comp'].min().groupby('homeTeam').first().nsmallest(20)
oboje = pd.concat([venku, doma], axis=1)
oboje = oboje.min(axis=1)
oboje.nsmallest(50)

Spartak Trnava                  2.0
Sparta Prague                   5.0
Sparta Praha                    5.0
Spartak Hradec Králové          9.0
Vítkovice                      13.0
Zbrojovka Brno                 13.0
Baník Ostrava                  13.0
Bohemians Prague               13.0
Slovan Bratislava              14.0
Dukla Prague                   16.0
Slovan UNV Bratislava          16.0
Červená Hviezda Bratislava     21.0
Viktoria Plzeň                 37.0
Slavia Prague                  61.0
Mladá Boleslav                 96.0
Teplice                       157.0
Slovan Liberec                157.0
dtype: float64

In [47]:
df[(df['competition'] == 'UCL') & (df['homeTeam'].str.contains('Slavia',na=False))].sort_values(by='remaining_comp').head(10)

,competition,date,score,homeTeam,homeTeamCountry,homeTeamUrl,awayTeam,awayTeamCountry,awayTeamUrl,location,phase,attendance,competitonOriginal,season,city,remaining_comp
211425,UCL,2007-12-12,0-3,Slavia Prague,Czech Republic,SK_Slavia_Prague,Sevilla,Spain,Sevilla_FC,"Stadion Evžena Rošického, Prague",groups,11689,2007–08 UEFA Champions League group stage,2007-08,Prague,37.0
211426,UCL,2007-12-12,0-3,Slavia Prague,Czech Republic,SK_Slavia_Prague,Sevilla,Spain,Sevilla_FC,"Stadion Evžena Rošického, Prague",groups,11689,2007–08 UEFA Champions League group stage,2007-08,Prague,37.0
211427,UCL,2007-12-12,0-3,Slavia Prague,Czech Republic,SK_Slavia_Prague,Sevilla,Spain,Sevilla_FC,"Stadion Evžena Rošického, Prague",groups,11689,2007–08 UEFA Champions League group stage,2007-08,Prague,37.0
211428,UCL,2007-12-12,0-3,Slavia Prague,Czech Republic,SK_Slavia_Prague,Sevilla,Spain,Sevilla_FC,"Stadion Evžena Rošického, Prague",groups,11689,2007–08 UEFA Champions League group stage,2007-08,Prague,37.0
211429,UCL,2007-12-12,0-3,Slavia Prague,Czech Republic,SK_Slavia_Prague,Sevilla,Spain,Sevilla_FC,"Stadion Evžena Rošického, Prague",groups,11689,2007–08 UEFA Champions League group stage,2007-08,Prague,37.0
211430,UCL,2007-12-12,0-3,Slavia Prague,Czech Republic,SK_Slavia_Prague,Sevilla,Spain,Sevilla_FC,"Stadion Evžena Rošického, Prague",groups,11689,2007–08 UEFA Champions League group stage,2007-08,Prague,37.0
211431,UCL,2007-12-12,0-3,Slavia Prague,Czech Republic,SK_Slavia_Prague,Sevilla,Spain,Sevilla_FC,"Stadion Evžena Rošického, Prague",groups,11689,2007–08 UEFA Champions League group stage,2007-08,Prague,37.0
211432,UCL,2007-12-12,0-3,Slavia Prague,Czech Republic,SK_Slavia_Prague,Sevilla,Spain,Sevilla_FC,"Stadion Evžena Rošického, Prague",groups,11689,2007–08 UEFA Champions League group stage,2007-08,Prague,37.0
413117,UCL,2019-11-27,1-3,Slavia Prague,Czech Republic,SK_Slavia_Prague,Inter Milan,Italy,Inter_Milan,"Sinobo Stadium, Prague",groups,1,2019–20 UEFA Champions League group stage,2019-20,Prague,47.0
413112,UCL,2019-11-27,1-3,Slavia Prague,Czech Republic,SK_Slavia_Prague,Inter Milan,Italy,Inter_Milan,"Sinobo Stadium, Prague",groups,1,2019–20 UEFA Champions League group stage,2019-20,Prague,47.0


In [44]:
east[east['phase'] == 'final'].tail(20)

,competition,date,score,homeTeam,homeTeamCountry,homeTeamUrl,awayTeam,awayTeamCountry,awayTeamUrl,location,phase,attendance,competitonOriginal,season,city,remaining_comp
1610,ICFC,1963-06-12,1-2,Dinamo Zagreb,Socialist Federal Republic of Yugoslavia,GNK_Dinamo_Zagreb,Valencia,Spain,Valencia_CF,"Stadion Maksimir, Zagreb",final,40000,1962–63 Inter-Cities Fairs Cup,1962-63,Zagreb,2.0
3614,ICFC,1968-09-11,0-0,Ferencváros,Hungary,Ferencv%C3%A1rosi_TC,Leeds United,England,Leeds_United_F.C.,"Népstadion, Budapest",final,76000.0,1967–68 Inter-Cities Fairs Cup,1967-68,Budapest,1.0
3615,ICFC,1968-09-11,0-0,Ferencváros,Hungary,Ferencv%C3%A1rosi_TC,Leeds United,England,Leeds_United_F.C.,"Népstadion, Budapest",final,76000.0,1967–68 Inter-Cities Fairs Cup,1967-68,Budapest,44.0
3616,ICFC,1968-09-11,0-0,Ferencváros,Hungary,Ferencv%C3%A1rosi_TC,Leeds United,England,Leeds_United_F.C.,"Népstadion, Budapest",final,76000.0,1967–68 Inter-Cities Fairs Cup,1967-68,Budapest,44.0
3617,ICFC,1968-09-11,0-0,Ferencváros,Hungary,Ferencv%C3%A1rosi_TC,Leeds United,England,Leeds_United_F.C.,"Népstadion, Budapest",final,76000.0,1967–68 Inter-Cities Fairs Cup,1967-68,Budapest,44.0
4156,CWC,1969-05-21,3-2,Slovan Bratislava,Czechoslovakia,%C5%A0K_Slovan_Bratislava,Barcelona,Spain,FC_Barcelona,"St. Jakob Stadium, Basel",final,19000.0,1968–69 European Cup Winners' Cup,1968-69,Basel,1.0
14619,CWC,1974-05-08,2-0,Magdeburg,East Germany,1._FC_Magdeburg,Milan,Italy,A.C._Milan,"De Kuip, Rotterdam",final,4641,1973–74 European Cup Winners' Cup,1973-74,Rotterdam,1.0
17065,CWC,1975-05-14,3-0,Dynamo Kyiv,Soviet Union,FC_Dynamo_Kyiv,Ferencváros,Hungary,Ferencv%C3%A1rosi_TC,"St. Jakob Stadium, Basel",final,13000.0,1974–75 European Cup Winners' Cup,1974-75,Basel,1.0
26685,EL,1979-05-09,1-1,Red Star Belgrade,Socialist Federal Republic of Yugoslavia,Red_Star_Belgrade,Borussia Mönchengladbach,West Germany,Borussia_M%C3%B6nchengladbach,"Red Star Stadium, Belgrade",final,64312,1978–79 UEFA Cup,1978-79,Belgrade,2.0
32928,CWC,1981-05-13,2-1,Dinamo Tbilisi,Soviet Union,FC_Dinamo_Tbilisi,FC Carl Zeiss Jena,East Germany,FC_Carl_Zeiss_Jena,"Rheinstadion, Düsseldorf",final,4750.0,1980–81 European Cup Winners' Cup,1980-81,Düsseldorf,1.0


In [ ]:
east[east['phase'].isin(serious)].groupby('location')['homeTeamUrl'].nunique().nlargest(50)

In [ ]:
east[east['phase'].isin(serious)].groupby('homeTeamCountry')['location'].nunique().nlargest(50)

In [ ]:
east[east['phase'].isin(serious)].groupby('city').size().nlargest(60)

In [ ]:
east[east['phase'].isin(serious)].groupby('location').size().nlargest(50)

In [ ]:
east[east['phase'] == 'Semi-finals'].tail(20)

In [ ]:
east[east['phase'] == 'Quarter-finals'].tail(20)

In [ ]:
df.groupby(['competition','phase'])['homeTeam'].nunique().head(250).tail(60)

In [ ]:
df[df['homeTeamCountry'] == 'Germany']['homeTeamUrl'].drop_duplicates()

In [ ]:
pd.Series(east[east['phase'] == 'quarter-finals'].groupby('season')['homeTeam'].nunique() / df[df['phase'] == 'quarter-finals'].groupby('season')['homeTeam'].nunique()).plot.bar(figsize=(25,5))

In [ ]:
df['competition']

In [ ]:
pd.Series(east[east['date'].dt.month.isin([1,2,3,4,5,6])].groupby('season')['homeTeam'].nunique() / df[df['date'].dt.month.isin([1,2,3,4,5,6])].groupby('season')['homeTeam'].nunique()).plot.bar(figsize=(25,5))

In [ ]:
pd.Series(east[(east['date'].dt.month.isin([1,2,3,4,5,6])) & (east['competition'] == 'UCL')].groupby('season')['homeTeam'].nunique() / df[(df['date'].dt.month.isin([1,2,3,4,5,6])) & (df['competition'] == 'UCL')].groupby('season')['homeTeam'].nunique()).plot.bar(figsize=(25,5))

In [ ]:
pd.Series(east[(east['date'].dt.month.isin([1,2,3,4,5,6])) & (east['competition'] == 'EL')].groupby('season')['homeTeam'].nunique() / df[(df['date'].dt.month.isin([1,2,3,4,5,6])) & (df['competition'] == 'EL')].groupby('season')['homeTeam'].nunique()).plot.bar(figsize=(25,5))

In [ ]:
pd.Series(east[(east['phase'].str.lower().str.contains('group',na=False)) & (east['competition'] == 'UCL')].groupby('season').size() / df[(df['phase'].str.lower().str.contains('group',na=False)) & (df['competition'] == 'UCL')].groupby('season').size()).plot.bar(figsize=(25,5))

In [ ]:
pd.Series(east[(east['date'].dt.month.isin([1,2,3,4,5,6])) & (east['competition'] == 'EL')].groupby('season')['homeTeam'].nunique() / df[(df['date'].dt.month.isin([1,2,3,4,5,6])) & (df['competition'] == 'EL')].groupby('season')['homeTeam'].nunique()).plot.bar(figsize=(25,5))

In [ ]:
pd.Series(four[(four['date'].dt.month.isin([1,2,3,4,5,6])) & (four['competition'] == 'UCL')].groupby('season')['homeTeam'].nunique() / df[(df['date'].dt.month.isin([1,2,3,4,5,6])) & (df['competition'] == 'UCL')].groupby('season')['homeTeam'].nunique()).plot.bar(figsize=(25,5))

In [ ]:
df[(df['date'].dt.month.isin([1,2,3,4,5,6])) & (df['competition'] == 'UCL')].groupby('season')['homeTeam'].nunique().tail(40)

In [ ]:
east[(east['date'].dt.month.isin([1,2,3,4,5,6])) & (east['competition'] == 'UCL')].groupby('season')['homeTeam'].nunique().tail(40)

In [ ]:
df[(df['date'].dt.month.isin([8,9,10])) & (df['homeTeamCountry'].isin(['Czechoslovakia','Czech Republic']))].groupby('season')['homeTeam'].nunique().tail(40)

In [ ]:
df[(df['date'].dt.month.isin([1,2,3,4,5,6])) & (df['homeTeamCountry'].isin(['Czechoslovakia','Czech Republic']))].groupby('season')['homeTeam'].nunique().tail(40)

In [ ]:
east[east['competition'] == 'UCL'].groupby('homeTeam').size().nlargest(16)

In [ ]:
df[df['homeTeamCountry'] == df['awayTeamCountry']].groupby(['competition','awayTeamCountry']).size().nlargest(20)

In [ ]:
east[east['homeTeamCountry'] == east['awayTeamCountry']]

## Albánie

In [ ]:
df[(df['homeTeamCountry'] == 'Czech Republic') & (df['competition'] == 'UCL')].groupby('season').size()